In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

In [ ]:
# Load and preprocess images
def load_data(base_path, class_folders, size=(64, 64)):
    X = []  # Grayscale images
    Y = []  # Color images

    for folder in class_folders:
        gray_folder = os.path.join(base_path, folder, 's1')
        color_folder = os.path.join(base_path, folder, 's2')

        if not os.path.exists(color_folder) or not os.path.exists(gray_folder):
            print(f"Skipping missing folder: {color_folder} or {gray_folder}")
            continue

        color_files = sorted(os.listdir(color_folder))
        gray_files = sorted(os.listdir(gray_folder))

        if len(color_files) != len(gray_files):
            print(f"Warning: Mismatched file counts in {folder}, skipping unmatched pairs.")
            continue

        for gray_file, color_file in zip(gray_files, color_files):
            color_path = os.path.join(color_folder, color_file)
            gray_path = os.path.join(gray_folder, gray_file)

            color_img = cv2.imread(color_path)
            gray_img = cv2.imread(gray_path, cv2.IMREAD_GRAYSCALE)

            if color_img is None:
                print(f"Failed to load color image: {color_path}")
                continue
            if gray_img is None:
                print(f"Failed to load grayscale image: {gray_path}")
                continue

            color_img = cv2.resize(color_img, size)
            gray_img = cv2.resize(gray_img, size)

            gray_img = np.expand_dims(gray_img, axis=-1)

            X.append(gray_img / 255.0)
            Y.append(color_img / 255.0)
            
    return np.array(X), np.array(Y)

base_path = '/kaggle/input/sentinel12-image-pairs-segregated-by-terrain/v_2'
class_folders = ['agri', 'barrenland', 'grassland', 'urban']
X, Y = load_data(base_path, class_folders)
X_train, X_val = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
Y_train, Y_val = Y[:int(0.8*len(Y))], Y[int(0.8*len(Y)):]

In [ ]:
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, Concatenate

# U-Net architecture for colorization
def build_colorization_model(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = UpSampling2D((2, 2))(c4)
    u5 = Concatenate()([u5, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = Concatenate()([u6, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = Concatenate()([u7, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = Conv2D(3, (1, 1), activation='sigmoid')(c7)  # 3 channels for RGB color

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
# Build and compile the model
input_shape = (64, 64, 1)  # (height, width, channels)
model = build_colorization_model(input_shape)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
# Training
try:
    history = model.fit(X_train, Y_train, batch_size=16, epochs=35, validation_data=(X_val, Y_val))
    weights_path = '/kaggle/working/colorization_model_weights.weights.h5'
    model.save_weights(weights_path)
    print(f"Model weights saved successfully to {weights_path}.")

except Exception as e:
    print(f"Error during training: {e}")

In [ ]:
# Evaluation
val_loss, val_accuracy = model.evaluate(X_val, Y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
# Visualisation
def test_model(model, X_val, Y_val, index=0):
    # Get the grayscale input and actual color image
    grayscale_img = X_val[index]
    true_color_img = Y_val[index]

    # Add batch dimension (1, 64, 64, 1) for model prediction
    grayscale_img_batch = np.expand_dims(grayscale_img, axis=0)

    # Predict the colorized image
    predicted_color_img = model.predict(grayscale_img_batch)[0]

    # Rescale from [0, 1] to [0, 255] for display purposes
    predicted_color_img = (predicted_color_img * 255).astype(np.uint8)
    true_color_img = (true_color_img * 255).astype(np.uint8)
    grayscale_img = (grayscale_img.squeeze() * 255).astype(np.uint8)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 3, 1)
    plt.title("Grayscale Input")
    plt.imshow(grayscale_img, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Predicted Color Image")
    plt.imshow(predicted_color_img)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("True Color Image")
    plt.imshow(true_color_img)
    plt.axis('off')

    plt.show()

test_model(model, X_val, Y_val, index=0)